In [1]:
import kafka
from kafka import KafkaClient, SimpleProducer
import tweepy

In [2]:
consumer_key = '****************************************************'
consumer_secret = '***********************************************' 
access_token = "***************************************************"
access_token_secret = "********************************************" 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [3]:
#override tweepy.StreamListener to add logic to on_status
client = KafkaClient("localhost:9092")
producer = SimpleProducer(client, async = True,
                          batch_send_every_n = 1000,
                          batch_send_every_t = 10)

#override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        try:
            producer.send_messages(b'test', status.text.encode('utf-8'))
        except Exception as e:
            print(e)
            return False
        return True
        
    def on_error(self, status_code):
        print(status_code)
        if status_code == 420:
            #returning False in on_data disconnects the stream
            print(status_code)
            return True
    def on_timeout(self):
        return True # Don't kill the stream

In [4]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [5]:
def start_stream():
    while True:
        try:
            myStream.filter(track=['apple', 'iphone8', 'iphone'], stall_warnings=True)
        except:
            continue 
start_stream()

KeyboardInterrupt: 

In [6]:
myStream.disconnect()

In [7]:
tweepy.__version__

'3.5.0'